# Quantum Teleportation

Alice and Bob are undercover KGBs in the United States. One night, they face danger. The day before, Alice and Bob have created a Bell Pair, 

$$
|A> \otimes |B> = \frac{1}{\sqrt{2}} (|0> \otimes |0> + |1> \otimes |1>),
$$

from qubits of their own, |A> and |B>, respectively. The Bell Pair is created utilizing a quantum circuit where a Hadamard gate is applied to Alice's qubit, followed by a CNOT gate, where Alice's qubit is the control qubit and Bob's qubit is the target. The function below achieves this:  

In [8]:
import projectq.setups.default
from projectq.ops import H, Measure, CNOT, X, Z, Rz
from projectq import MainEngine
from projectq.backends import CircuitDrawer
from projectq.meta import Dagger, Control

def create_bell_pair(eng):
    a = eng.allocate_qubit()
    b = eng.allocate_qubit()

    H | a
    CNOT | (a, b)

    return a, b

Below, the circuit for Bell Pair creation is coded and portrayed:

In [20]:
# create a main compiler engine
drawing_engine = CircuitDrawer()
eng = MainEngine(drawing_engine)

create_bell_pair(eng)

eng.flush()

![alt text](bellpair.jpg "Bell Pair circuit")

Alice and Bob hold tight onto her and his half of the Bell Pair. On this particular night though, Alice is kidnapped by another foreign entity. Bob can't find her, but he knows if Alice is alive, she will send a signal to him using the Bell Pair and an additional qubit $|\psi>$, in which it's state can be used to provide her location. What mechanism must now be used for Alice to communicate the qubit $|\psi>$ to Bob?

One mechanism is Quantum Teleportation. To achieve, a sequence of steps must be followed by both Alice and Bob. First, Alice must entangle her half of the Bell Pair with the qubit $|\psi>$. Following that, she must measure both qubits, her half of the Bell Pair and qubit $|\psi>$. Then, she must send those measurements to Bob, where Bob will use those measurements to determine what actions he must take to gain information about the state of the qubit $|\psi>$. In this case, based on the measurements Bob receives, he will have to do 1 of 4 things, through the use of an X gate and/or Z gate. If he receives a measurement of 00, he won't have to do anything to his own half of the Bell Pair. His half already represents the state $|\psi>$. If he receives a measurement of 01, he will have to apply an X gate to his own half of the Bell Pair. If he receives a measurement of 10, he will have to apply a Z gate to his own half of the Bell Pair. Last, if he receives a measurement of 11, he will have to first apply a X gate, followed by a Z gate. 

The full implementation and diagram for the circuit is provided below:

In [19]:
def run_teleport(eng, state_creation_function, verbose=False):
    # make a Bell-pair
    b1, b2 = create_bell_pair(eng)

    # Alice creates a nice state to send
    psi = eng.allocate_qubit()
    if verbose:
        print("Alice is creating her state from scratch, i.e., |0>.")
    state_creation_function(eng, psi)

    # entangle it with Alice's b1
    CNOT | (psi, b1)
    if verbose:
        print("Alice entangled her qubit with her share of the Bell-pair.")

    # measure two values (once in Hadamard basis) and send the bits to Bob
    H | psi
    Measure | (psi, b1)
    msg_to_bob = [int(psi), int(b1)]
    if verbose:
        print("Alice is sending the message {} to Bob.".format(msg_to_bob))

    # Bob may have to apply up to two operation depending on the message sent
    # by Alice:
    with Control(eng, b1):
        X | b2
    with Control(eng, psi):
        Z | b2

    # try to uncompute the psi state
    if verbose:
        print("Bob is trying to uncompute the state.")
    with Dagger(eng):
        state_creation_function(eng, b2)

    # check whether the uncompute was successful. The simulator only allows to
    # delete qubits which are in a computational basis state.
    del b2
    eng.flush()

    if verbose:
        print("Bob successfully arrived at |0>")


if __name__ == "__main__":
    # create a main compiler engine with a simulator backend:
    eng = MainEngine()

    # define our state-creation routine, which transforms a |0> to the state
    # we would like to send. Bob can then try to uncompute it and, if he
    # arrives back at |0>, we know that the teleportation worked.
    def create_state(eng, qb):
        H | qb
        Rz(1.21) | qb

    # run the teleport and then, let Bob try to uncompute his qubit:
    run_teleport(eng, create_state, verbose=False)

(Note: This is the (slow) Python simulator.)


![alt text](QuantumTeleport.jpg "Quantum Teleport circuit")